# Find the Most Visited Floor

In [0]:
%sql
create table entries ( 
name varchar(20),
address varchar(20),
email varchar(20),
floor int,
resources varchar(10));

insert into entries 
values ('A','Bangalore','A@gmail.com',1,'CPU'),('A','Bangalore','A1@gmail.com',1,'CPU'),('A','Bangalore','A2@gmail.com',2,'DESKTOP')
,('B','Bangalore','B@gmail.com',2,'DESKTOP'),('B','Bangalore','B1@gmail.com',2,'DESKTOP'),('B','Bangalore','B2@gmail.com',1,'MONITOR');

select * from entries;

name,address,email,floor,resources
A,Bangalore,A@gmail.com,1,CPU
A,Bangalore,A1@gmail.com,1,CPU
A,Bangalore,A2@gmail.com,2,DESKTOP
B,Bangalore,B@gmail.com,2,DESKTOP
B,Bangalore,B1@gmail.com,2,DESKTOP
B,Bangalore,B2@gmail.com,1,MONITOR


In [0]:
%sql


select
name,
floor,
count(1) as total_visits,
rank() over(partition by name order by count(1) desc) as rn
from entries
group by name,floor;

name,floor,total_visits,rn
A,1,2,1
A,2,1,2
B,2,2,1
B,1,1,2


In [0]:
%sql
select
name,
count(1) as total_visits,
string_agg(resources,',') as resources_used
from entries
group by name

name,total_visits,resources_used
A,3,"CPU,CPU,DESKTOP"
B,3,"DESKTOP,DESKTOP,MONITOR"


In [0]:
%sql
with distinct_records as (
  select distinct name, resources from entries
),
agg_resources as (
  select
    name,
    string_agg(resources, ',') as used_resources
  from distinct_records
  group by name
)
select * from agg_resources;

name,used_resources
A,"CPU,DESKTOP"
B,"DESKTOP,MONITOR"


In [0]:
%sql
with distinct_records as (
  select distinct name, resources from entries
),
agg_resources as (
  select
    name,
    string_agg(resources, ',') as used_resources
  from distinct_records
  group by name
),
total_visits as (
  select
    name,
    count(1) as total_visits,
    string_agg(resources, ',') as resources_used
  from entries
  group by name
),
floor_visits as (
  select
    name,
    floor,
    count(1) as total_visits,
    rank() over(partition by name order by count(1) desc) as rn
  from entries
  group by name, floor
)
select 
  tv.name as name,
  tv.total_visits as total_visits,
  fv.floor as most_visited_floor,
  ar.used_resources as used_resources
from total_visits tv
join floor_visits fv on tv.name = fv.name
join agg_resources ar on tv.name = ar.name
where fv.rn = 1;

name,total_visits,most_visited_floor,floor_visits,used_resources
A,3,1,2,"DESKTOP,CPU"
B,3,2,2,"MONITOR,DESKTOP"
